Load the environment variables from a .env file

To get an api key, navigate to https://aistudio.google.com/apikey to create one. Then create a .env file in the same directory as this notebook with the following content:
```GOOGLE_API_KEY=your_api_key_here```


In [1]:
from google import genai
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
client = genai.Client(api_key=api_key)

Single shot question answering with Gemini 2.0 Flash to check correct working of the code

In [2]:
from pydantic import BaseModel


class Recipe(BaseModel):
    medicijn: str
    categorie: str
    recept: str
    medische_vraag: str
    patientinformatie: str



Create a function to get a recipe based on a medical question

In [3]:
import json
import re


def get_recipe(medical_question: str) -> Recipe:
    prompt = (
        "Je bent een medisch expert. Beantwoord de medische vraag met een recept volgens het volgende template: "
        "'Rp. [Medicijnnaam] [Sterkte] [Vorm]\ndtd. No. [Aantal]\n S. [Instructie]'. "
        "Neem relevante informatie over de patiënt in acht, zoals leeftijd en eventuele allergieën. "
        "Geef de formele medicijnnaam in het JSON-veld 'medicijn'. "
        "Benoem de categorie van het medicijn in het JSON-veld 'categorie'. "
        "Geef bij de instructie ook de duur van de behandeling aan. "
        "Herhaal de medische vraag in het JSON-veld 'medische_vraag'. "
        "Voeg de patiëntinformatie toe in het JSON-veld 'patientinformatie'. "
    )

    response = client.models.generate_content(
        model="gemini-2.0-flash-001",
        contents=medical_question,
        config={
            "response_mime_type": "application/json",
            "response_schema": Recipe,
            "system_instruction": prompt,
        })

    try:
        parsed = extract_json_from_text(response.text)
        return Recipe.model_validate(parsed)
    except Exception as e:
        print("Fout bij het parsen van het recept:", e)
        raise


def extract_json_from_text(text: str) -> dict:
    match = re.search(r'\{.*?\}', text, re.DOTALL)
    if match:
        return json.loads(match.group())
    else:
        raise ValueError("Geen JSON gevonden in het antwoord")


question =  """ vraag: "Ik heb een keelontsteking. Heb ik antibiotica nodig?",
             "patientinformatie":
                 "leeftijd": 34,
                 "geslacht": "vrouw",
                 "koorts": True,
                 "duur_klachten_dagen": 3,
                 "pijn_bij_slikken": True,
                 "tonsillen_met_pus": True,
                 "hoesten": False,
                 "penicilline_allergie": False
             """
recipe = get_recipe(question)
print(recipe)


medicijn='Amoxicilline' categorie='Antibioticum' recept='Rp. Amoxicilline 500mg capsules\ndtd. No. 21\nS. Driemaal daags 1 capsule gedurende 7 dagen.' medische_vraag='Ik heb een keelontsteking. Heb ik antibiotica nodig?' patientinformatie='leeftijd: 34, geslacht: vrouw, koorts: True, duur_klachten_dagen: 3, pijn_bij_slikken: True, tonsillen_met_pus: True, hoesten: False, penicilline_allergie: False'


In [4]:
import pandas as pd

filepath = "../data/Z_unieke_vragen_deel_2.json"
questions = pd.read_json(filepath)

questions

,vraag,patientinformatie
0,Ik heb een lichte tandvleesontsteking. Kan ik ...,"{'leeftijd': 69, 'geslacht': 'vrouw', 'chronis..."
1,"Ik heb Chlamydia, welke behandeling is nodig?","{'leeftijd': 31, 'geslacht': 'vrouw', 'chronis..."
2,Ik heb krentenbaard (impetigo). Wat kan ik geb...,{'leeftijd': 3}
3,Mijn kind heeft dubbele oorontsteking. Welke d...,"{'leeftijd': 5, 'geslacht': 'man', 'chronische..."
4,Ik heb een bacteriele ooginfectie. Er is indic...,"{'leeftijd': 57, 'geslacht': 'vrouw', 'penicil..."
5,Ik heb een steenpuist. Er is indicatie voor an...,"{'leeftijd': 50, 'geslacht': 'vrouw', 'chronis..."
6,"Ik heb een longontsteking, ik moet antibiotica...","{'leeftijd': 82, 'geslacht': 'man', 'chronisch..."
7,Ik heb traag werkende schildklier. Moet ik lev...,"{'leeftijd': 34, 'geslacht': 'vrouw', 'chronis..."
8,Ik heb sinusitis. Ik moet antibiotica krijgen....,"{'leeftijd': 47, 'geslacht': 'vrouw', 'chronis..."
9,Ik heb een vaginale infectie. Wat helpt?,"{'leeftijd': 50, 'geslacht': 'vrouw', 'chronis..."


In [5]:
import json
import pandas as pd

recipes = []

filepath = "../data/Z_unieke_vragen_deel_2.json"
questions = pd.read_json(filepath)

for idx, row in questions.iterrows():
    medical_context = {
        "vraag": row["vraag"],
        "patientinformatie": row["patientinformatie"],
    }
    try:
        medical_context_str = json.dumps(medical_context, ensure_ascii=False)
        recipe = get_recipe(medical_context_str)
        recipes.append(recipe)

        print(f"Recipe gegenereerd: {recipe.model_dump()}")
    except Exception as e:
        print(f"Fout bij het genereren van een recept voor deze vraag: {medical_context}")
        print(f"Error: {e}")

output_file = "../data/generated_recipes_deel_2.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump([recipe.model_dump() for recipe in recipes], f, ensure_ascii=False, indent=4)

Recipe gegenereerd: {'medicijn': 'Chloorhexidine', 'categorie': 'Antisepticum', 'recept': 'Rp. Chloorhexidine 0,2% mondspoeling\ndtd. No. 1\nS. 2 maal daags gedurende 2 weken spoelen na het tandenpoetsen', 'medische_vraag': 'Ik heb een lichte tandvleesontsteking. Kan ik hier iets voor slikken?', 'patientinformatie': 'patiënt is een 69-jarige vrouw zonder chronische ziekten of penicilline-allergie en heeft geen koorts en is niet ziek'}
Recipe gegenereerd: {'medicijn': 'Azitromycine', 'categorie': 'Antibioticum', 'recept': 'Rp. Azitromycine 1g Tablet\ndtd. No. 1\nS. Eenmalig 1 gram oraal innemen.', 'medische_vraag': 'Ik heb Chlamydia, welke behandeling is nodig?', 'patientinformatie': 'Patiënt is een 31-jarige vrouw met diabetes en geen penicilline-allergie.'}
Recipe gegenereerd: {'medicijn': 'Fusidinezuur', 'categorie': 'Antibioticum', 'recept': 'Rp. Fusidinezuur 20mg/g zalf\ndtd. No. 1 tube\nS. 3 maal daags dun aanbrengen gedurende 7 dagen', 'medische_vraag': 'Ik heb krentenbaard (impe

Check each recipe for correctness. Correctness is defined as follows:
- The recipe should contain a medicine name, strength, form, number of doses, and instructions.
- The medicine name should be a valid medicine name.
- The strength should be a valid strength.
- The form should be a valid form (e.g., tablet, capsule, liquid).
- The number of doses should be a valid number.
- The instructions should include the duration of the treatment.
- The indication should be relevant to the medical question.
